In [1]:
#imports
using Distributions;
using Random;
using DataFrames;
using CSV;
using Statistics;
using LinearAlgebra;
using Flux;
using Flux: params, 
            Dense, 
            Chain, 
            glorot_normal, 
            normalise, 
            Optimiser,
            train!;


function nn_estimation(file_name,file_name1,file_name2,batch_samples)    
    
    #parameters of the function
    r = Float16(0.05);
    T = Int(1);  #ttm
    N = Int(1);  #number of evolution steps
    K = Int(100); #strike
    mu = Float16(-0.05); 

    d = Int(100); #number of dimensions
    #batch_samples = Int(800); #total samples in batch
    a = 90;
    b = 110;
    initial_sampler = Uniform(a,b);
    
    time_grid = LinRange(1/d, 1, d); 
    sigma = 0.1 .+ 0.5 .* time_grid; #define changing variance on the time grid

    mc_samples = Int(500000); #number of samples to take for Monte-Carlo approximation
    mc_exp_rounds = Int(1); #number of times to repeat MC for the error average
    
    learning_rate = Float16(0.001); #initial learning rate
    learn_rate_decrease = 250000; #how frequently to decay learning rate

    train_steps = Int(200000); #total number of training epochs
    err_step = Int(25000); #after how many training steps to compare errors

    #placeholder
    t_mc = 0
    
    function x_sde(X::Array) #discretisation of SDE 
        #how_many_samples = length(X)
        for _ in 1:N
            #take appropriate samples from normal 
            eps = rand(Normal(0,1),(d, batch_samples))
            X = X .* (exp.((mu .- 0.5*sigma.^2)*(T/N) .+  sqrt(T/N).*( sigma .* eps)))
       # println(eps)
        end
        return X
    end; 

    function x_phi(x::Array) #function to use with FK expectation /initial conditions
        phi_(x) = exp(-r*T) * max((maximum(x)-K),0)
        mapslices(phi_, x; dims =1)
    end;
    
    #standard normalisation function 
    function norm_ab(y)
        mid_point = (a + b) / 2
        y_norm(y) = (y .- mid_point) ./ (b-a)
        return mapslices(y_norm,y;dims =1)
    end
    
    #initialise error file and create row headers
    #three separate holders for errors
    df_row_each = DataFrame(step = "step",
                        l1_errs="l1_errs",l2_errs="l2_errs",li_errs="li_errs",
                        rel_l1_errs="rel_l1_errs",rel_l2_errs="rel_l2_errs",rel_li_errs="rel_li_errs")

    df_row = DataFrame(step = "step",
                        l1_errs="l1_errs",
                        rel_l1_errs="rel_l1_errs")
    
    CSV.write(file_name, df_row_each, append = true);

    CSV.write(file_name1, df_row, append = true);

    CSV.write(file_name2, df_row, append = true);

    
    #calculate errors and write to file
    function k_iter_output(k)
        
        #generate test batch
        X_test = rand(initial_sampler,(d,batch_samples))
        X_test_norm = norm_ab(X_test)
        
        #generate mc data
        function mc_sampler(X_test)
            x_mc_store = zeros((1,batch_samples))
            for _ in 1:mc_samples
                x_mc_store += Array((x_phi(x_sde(X_init))))
            end
            phi_mc = x_mc_store ./ mc_samples;
            return Array(phi_mc)
        end
                   
        #initial errors for finding the mean
#        l1_errs = 0.
#        rel_l1_errs = 0.

        l1_errs,l2_errs,li_errs = 0.,0.,0.
        rel_l1_errs, rel_l2_errs, rel_li_errs = 0., 0., 0.

        l1_errs1 = 0.
        rel_l1_errs1 = 0.
        
        l1_errs2 = 0.
        rel_l1_errs2 = 0.
        
        #run through testmode NN for comparison
        testmode!(m)
        u_i = m(X_test_norm)
        
        #run through testmode NN for comparison
        testmode!(m1)
        u_i1 = m1(X_test_norm)

        testmode!(m2)
        u_i2 = m2(X_test_norm)

        for _ in 1:mc_exp_rounds

            #take mc samples
            t_start = time()
            mc_i = mc_sampler(X_test)
            t_end = time()
            t_mc += t_end - t_start
            u_ref = abs.(max.(mc_i,1e-8))
            
            #calculate and output errors
            errs = vec(abs.(u_i - mc_i))
            l1_errs += mean(errs)
            rel_errs = errs ./ u_ref
            rel_l1_errs += mean(rel_errs)
            l2_errs += mean(errs.^2)
            li_errs = max(li_errs, maximum(errs))
            rel_l2_errs += mean(rel_errs.^2)    
            rel_li_errs = max(rel_li_errs,maximum(rel_errs))

            errs1 = vec(abs.(u_i1 - mc_i))
            l1_errs1 += mean(errs1)
            rel_errs1 = errs1 ./ u_ref
            rel_l1_errs1 += mean(rel_errs1)

            errs2 = vec(abs.(u_i2 - mc_i))
            l1_errs2 += mean(errs2)
            rel_errs2 = errs2 ./ u_ref
            rel_l1_errs2 += mean(rel_errs2)
            
        end

        #find means
#        l1_errs = l1_errs/mc_exp_rounds
#        rel_l1_errs= rel_l1_errs/mc_exp_rounds  
        l1_errs,l2_errs = l1_errs/mc_exp_rounds, sqrt(l2_errs/mc_exp_rounds)
        rel_l1_errs,rel_l2_errs = rel_l1_errs/mc_exp_rounds, sqrt(rel_l2_errs/mc_exp_rounds)  
                    
        l1_errs1 = l1_errs1/mc_exp_rounds
        rel_l1_errs1 = rel_l1_errs1/mc_exp_rounds  

        l1_errs2 = l1_errs2/mc_exp_rounds
        rel_l1_errs2 = rel_l1_errs2/mc_exp_rounds  
        
        #write to file
#        df_row = DataFrame(step = k,
#                            l1_errs=l1_errs,
#                            rel_l1_errs=rel_l1_errs)
#
        #write to file
        df_row = DataFrame(step = k,
                            l1_errs=l1_errs,l2_errs=l2_errs,li_errs=li_errs,
                            rel_l1_errs=rel_l1_errs,rel_l2_errs=rel_l2_errs,
                            rel_li_errs=rel_li_errs)

        df_row1 = DataFrame(step = k,
                    l1_errs=l1_errs1,
                    rel_l1_errs=rel_l1_errs1)
        
        df_row2 = DataFrame(step = k,
                    l1_errs=l1_errs2,
                    rel_l1_errs=rel_l1_errs2)

        CSV.write(file_name, df_row, append = true)
        CSV.write(file_name1, df_row1, append = true)
        CSV.write(file_name2, df_row2, append = true)

    end
    
    function generate_training_data(X_init,x_sde,x_phi)    
        X_0 = Array(norm_ab(X_init))
        X_sde = x_sde(X_init)
        y_train = x_phi(X_sde)
        return [(X_0,y_train)]
    end

    #define network layers
    input = Dense(d, d + d, tanh; 
                           bias = false, 
                           init = glorot_normal)

    hidden = Dense(d + d, d + d, tanh;
                            bias = false,
                            init = glorot_normal)

    #no activation on the last layer
    output = Dense(d + d,1,identity)
    
    #define network layers
    input1 = Dense(d, d + d, tanh; 
                           bias = false, 
                           init = glorot_normal)

    hidden1 = Dense(d + d, d + d, tanh;
                            bias = false,
                            init = glorot_normal)

    #no activation on the last layer
    output1 = Dense(d + d,1,identity)
    
    #define network layers
    input2 = Dense(d, d + d, tanh; 
                           bias = false, 
                           init = glorot_normal)

    hidden2 = Dense(d + d, d + d, tanh;
                            bias = false,
                            init = glorot_normal)

    #no activation on the last layer
    output2 = Dense(d + d,1,identity)

    
    #define network architecture
    m = Chain(input,
        #        batch_norm_layer,
                hidden,
        #        batch_norm_layer,
        #        hidden,
        #        batch_norm_layer,
                output)
    
    #define network architecture
    m1 = Chain(input1,
        #        batch_norm_layer,
                hidden1,
        #        batch_norm_layer,
        #        hidden,
        #        batch_norm_layer,
                output1)
        
    #define network architecture
    m2 = Chain(input2,
        #        batch_norm_layer,
                hidden2,
        #        batch_norm_layer,
        #        hidden,
        #        batch_norm_layer,
                output2)
    
   #loss function = 
    loss(u,v) = mean((m(u) - v).^2)
    loss1(u,v) = mean((m1(u) - v).^2)
    loss2(u,v) = mean((m2(u) - v).^2)
    
    ps = Flux.params(m)
    ps1 = Flux.params(m1)
    ps2 = Flux.params(m2)

    opt = Optimiser(ExpDecay(learning_rate,0.1,learn_rate_decrease,1e-8),ADAM()) #optimiser

    #set to train mode
    trainmode!(m)
    trainmode!(m1)
    trainmode!(m2)
    
    #generate initial training data
    X_init = rand(initial_sampler,(d,batch_samples))
    data = generate_training_data(X_init,x_sde,x_phi)

    X_init_0 = X_init
    
    data1 = data
    data2 = data
    
    k_iter_output(0) #compare with MC at this stage

    #start training time counter
    t_nn_start = time()
    
    for k in 1:train_steps
        
        #generate new training data each epoch
        X_init = rand(initial_sampler,(d,batch_samples))
        data = generate_training_data(X_init,x_sde,x_phi)
        
        #re.evolve SDE at current X
        data2 = generate_training_data(X_init_0,x_sde,x_phi)
        
        #learning step
        train!(loss,ps,data,opt)
        train!(loss1,ps1,data1,opt)
        train!(loss2,ps2,data2,opt)
        
        #output the errors and timings at these steps
        if mod(k,err_step) == 0 
            
            t_nn_end = time()
            t_nn = t_nn_end - t_nn_start #timer for the training steps
            k_iter_output(k) #compare with MC at this stage
            t_nn_start = time() #start new training timer
            trainmode!(m) #set back train mode
            trainmode!(m1) #set back train mode
            trainmode!(m2) #set back train mode
            
            #reset x and fix phi every 25k steps
            X_init_0 = rand(initial_sampler,(d,batch_samples))
            data1 = generate_training_data(X_init_0,x_sde,x_phi)

        end
        

    end
    
    print("Output ready!")

end

nn_estimation("gbm_eachx_d100_batch400.csv","gbm_onex_d100_batch400.csv","gbm_onex_eachphi_d100_batch400.csv",400);
nn_estimation("gbm_eachx_d100_batch100.csv","gbm_onex_d100_batch100.csv","gbm_onex_eachphi_d100_batch100.csv",100);



Output ready!Output ready!

In [ ]:
nn_estimation("gbm_eachx_d100_batch1000.csv","gbm_onex_d100_batch1000.csv","gbm_onex_eachphi_d100_batch1000.csv",1000);
